<a href="https://colab.research.google.com/github/vsaathwika0904/hds5210-2023/blob/main/midterm/midterm-2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [27]:
def priest(sex, age, respirateRt, oxSaturation, hrtrate, bp, temperature, alertness, insOxygen, perfrmsts):

    """

    (String, float, float, float, float, float, float, String, String, String) -> float)

    The clinical Severity Score 'PRIEST' COVID-19's
    risk percentage is calculated using the priest method.

    Sex, alertness, insOxygen, and performance arguments are accepted by the priest function in String; all other arguments are accepted in float.

    >>> priest("FEMALE",80,23,0.96,105,115,38.1,"ALERT","supplemental oxygen","limited activity, can self-care")
    0.34
    >>> priest("Male",16,10,0.96,105,115,38.1,"ALERT","AIR","unrestricted normal activity")
    0.03

    >>> priest("FEMALE",18,23,0.96,135,115,38.1,"ALERT","AIR","unrestricted normal activity")
    0.15

    """


    prscr = 0

    if sex.lower()=="male":
        prscr = prscr + 1
    elif sex.lower()=="female":
        prscr = prscr + 0

    if age>=16 and age<=49:
        prscr = prscr + 0
    elif age >= 50 and age<=65:
        prscr = prscr + 2
    elif age >= 66 and age<=80:
        prscr = prscr + 3
    elif age > 80:
        prscr = prscr + 4


    if alertness.lower() == "confused or not alert":
        prscr = prscr +  3

    if perfrmsts.lower() == "limited self-care":
        prscr = prscr +  3
    elif perfrmsts.lower() == "limited activity, can self-care":
        prscr = prscr +  2
    if perfrmsts.lower() == "limited strenuous activity, can do light activity":
        prscr = prscr +  1
    elif perfrmsts.lower() == "bed/chair bound, no self-care":
        prscr = prscr +  4

    if insOxygen.lower() == "supplemental oxygen":
        prscr = prscr +  2
    elif insOxygen.lower() == "air":
        prscr = prscr +  0

    if respirateRt < 9:
        prscr = prscr +  3
    if respirateRt > 24:
        prscr = prscr +  3
    elif respirateRt >= 21 and respirateRt <= 24:
        prscr = prscr +  2
    elif respirateRt >= 9 and respirateRt <= 11:
        prscr = prscr +  1

    if oxSaturation < 0.92:
        prscr = prscr +  3
    elif oxSaturation >= 0.94 and oxSaturation <= 0.95:
        prscr = prscr +  1
    elif oxSaturation >= 0.92 and oxSaturation <= 0.93:
        prscr = prscr +  2

    if hrtrate > 130:
        prscr = prscr +  3
    elif hrtrate >= 111 and hrtrate <= 130:
        prscr = prscr +  2
    elif hrtrate >= 91 and hrtrate <= 110:
        prscr = prscr +  1
    elif hrtrate < 41:
        prscr = prscr +  3
    elif hrtrate >= 41 and hrtrate <= 50:
        prscr = prscr +  1


    if bp > 219:
        prscr = prscr +  3
    elif bp >= 101 and bp <= 110:
        prscr = prscr +  1
    elif bp >= 91 and bp <= 100:
        prscr = prscr +  2
    elif bp < 91:
        prscr = prscr +  3

    if temperature > 39.0:
        prscr = prscr +  2
    elif temperature >= 38.1 and temperature <= 39.0:
        prscr = prscr +  1
    if temperature < 35.1:
        prscr = prscr +  3
    elif temperature >= 35.1 and temperature <= 36.0:
        prscr = prscr +  1


    percscale = [0.01, 0.01, 0.02, 0.02, 0.03, 0.09, 0.15, 0.18, 0.22, 0.26, 0.29, 0.34, 0.38, 0.46, 0.47, 0.49, 0.55, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,0.99]
    if prscr < 26:
        return percscale[prscr]

    return 0.99

In [28]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("FEMALE",80,23,0.96,105,115,38.1,"ALERT","supplemental oxygen","limited activity, can self-care")
Expecting:
    0.34
ok
Trying:
    priest("Male",16,10,0.96,105,115,38.1,"ALERT","AIR","unrestricted normal activity")
Expecting:
    0.03
ok
Trying:
    priest("FEMALE",18,23,0.96,135,115,38.1,"ALERT","AIR","unrestricted normal activity")
Expecting:
    0.15
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [29]:
import requests, json, csv

def find_hospital(age, sex, risk_pct):
    """

    (integer, String, float) -> String

    The find_hospital() method locates the appropriate hospital for a patient
    based on a number of variables, including riskPercentage, age, and sex.


    Age, sex, and riskPerc are all integers, while sex is a string.
    The find_hospital() method accepts these three arguments.
    returns the output hospital's name as a String.


    >>> find_hospital(80, "FEMALE", 0.34)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(16, "MALE", 0.03)
    'Select Specialty Hospital - Northeast Atlanta'

    >>> find_hospital(18, "FEMALE", 0.15)
    'Select Specialty Hospital - Northeast Atlanta'

    """


    url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd"

    params={"age":age,"sex":sex.lower(),"risk_pct":risk_pct}

    specifiedHosp = requests.get(url, params=params)

    return (specifiedHosp.json())["hospital"]

In [30]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(80, "FEMALE", 0.34)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(16, "MALE", 0.03)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
**********************************************************************
File "__main__", line 20, in NoName
Failed example:
    find_hospital(16, "MALE", 0.03)
Expected:
    'Select Specialty Hospital - Northeast Atlanta'
Got:
    'Childrens Healthcare of Atlanta at Scottish Rite'
Trying:
    find_hospital(18, "FEMALE", 0.15)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [31]:
def get_address(Hospitalname):
    """
    (string)->string

    the get_address command using the provided url, locate the hospital's address based on the hospital name.

    Get_address function finds address by using API, where it has hospital name json data in it.
    It expects hospital name as String and returns address as String.

    >>> get_address("Wesley Woods Geriatric Hospital")
    '1821 CLIFTON ROAD, NE'

    >>> get_address("Childrens Healthcare of Atlanta at Scottish Rite")
    '1001 JOHNSON FERRY RD'

    >>> get_address("Select Specialty Hospital - Northeast Atlanta")
    '1821 CLIFTON ROAD NE'

    """

    respons = requests.get("https://hds5210-data.s3.amazonaws.com/ga_hospitals.json")

    response = respons.json()


    if Hospitalname.upper() in response:
        return response[Hospitalname.upper()]['ADDRESS']
    else:
        return "Not able to find"

In [32]:
get_address("Wesley Woods Geriatric Hospital")

'1821 CLIFTON ROAD, NE'

In [33]:
get_address("Childrens Healthcare of Atlanta at Scottish Rite")

'1001 JOHNSON FERRY RD'

In [34]:
get_address("Select Specialty Hospital - Northeast Atlanta")

'1821 CLIFTON ROAD NE'

In [35]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("Wesley Woods Geriatric Hospital")
Expecting:
    '1821 CLIFTON ROAD, NE'
ok
Trying:
    get_address("Childrens Healthcare of Atlanta at Scottish Rite")
Expecting:
    '1001 JOHNSON FERRY RD'
ok
Trying:
    get_address("Select Specialty Hospital - Northeast Atlanta")
Expecting:
    '1821 CLIFTON ROAD NE'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [36]:
def process_people(url):
    """
    (String) -> Dictionary

    All patient information, including risk, hospital,
    and hospital address, will be generated by this process_people method
    in a dictionary (in JSON format).

    This process_people method accepts url arguments and
    outputs a Dictionary object containing all patient data.

    """

    urlContent = requests.get(url)

    rowsWithDel = urlContent.text.split("\n")

    xs = csv.reader(rowsWithDel, delimiter='|')

    next(xs)

    itr = iter(xs)

    dictionaryPatients = {}
    try:
        while 1:
            value = next(itr)

            if all(field == '' for field in value):
                break

            patient, sex, age, rspRt, oxSt, htRate, sBP, temp, alertness, insOx, status = value[:11]


            risk = priest(sex, float(age), float(rspRt), float(oxSt), float(htRate), float(sBP), float(temp), alertness, insOx, status)

            Hptl = find_hospital(int(age), sex, risk)


            dictionaryPatients[patient] = [sex, float(age), float(rspRt), float(oxSt), float(htRate), float(sBP), float(temp), alertness, insOx, status, risk, Hptl, get_address(Hptl)]

    except StopIteration:
        return dictionaryPatients


    return dictionaryPatients
    print(process_people("https://hds5210-data.s3.amazonaws.com/people.psv"))

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [37]:
"""

It determines whether the information in the url
and the result produced by process_people are equivalent.

"""
methodCalc = process_people("https://hds5210-data.s3.amazonaws.com/people.psv")

methodCalcStr = json.loads(json.dumps(methodCalc))


original = requests.get("https://hds5210-data.s3.amazonaws.com/people_results.json")

originalstr = json.loads(json.dumps(original.json()))



if methodCalcStr == originalstr:
    print("Results Are Same")
else:
    print("Results are Not Same")

Results Are Same


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---